In [61]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import os
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
from keras.layers import Activation
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from tensorflow.keras.layers import *
import pandas as pd
from PIL import Image
import pickle
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from imblearn.over_sampling import SMOTE

In [48]:
images_folder_path = 'train/images/'
labels_csv_path="train/train.csv"

In [49]:
labels_df=pd.read_csv(labels_csv_path)

In [50]:
labels_df.head()

,image_id,filename,label
0,1,1.jpg,2
1,2,2.jpg,4
2,3,3.jpg,2
3,4,4.jpg,3
4,5,5.jpg,5


In [51]:
labels_df["label"].value_counts()

label
2    2349
4    2079
5    1185
6     882
3     534
1     171
Name: count, dtype: int64

In [52]:
car_damage_type={1:"crack",2:"scratch", 3:"tire flat", 4:"dent",5:"glass shatter",6:"lamp broken"}

In [53]:
labels_df['label'] = labels_df['label'].map(car_damage_type)

In [54]:
labels_df["label"].value_counts()

label
scratch          2349
dent             2079
glass shatter    1185
lamp broken       882
tire flat         534
crack             171
Name: count, dtype: int64

In [56]:
X_train=[]

In [58]:
image_size=500

In [59]:
def resize_keep_aspect_ratio(img,desired_size):
    height, width = img.shape[:2]
    max_dim = max(height, width)
    ratio = desired_size / max_dim
    new_size = tuple([int(x * ratio) for x in (width, height)])
    resized_img = cv2.resize(img, (new_size[0], new_size[1]))
    pad_w = desired_size - new_size[0]
    pad_h = desired_size - new_size[1]
    top, bottom = pad_h // 2, pad_h - (pad_h // 2)
    left, right = pad_w // 2, pad_w - (pad_w // 2)
    img=cv2.copyMakeBorder(resized_img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=[0, 0, 0])
    return img

In [62]:
def SMOTE_Data(train_df=labels_df):

    X_train = []
    for img in train_df['filename']:
        img = cv2.imread(os.path.join(images_folder_path,img))
        img=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img=resize_keep_aspect_ratio(img,image_size)
        X_train.append(img)
        
    print(np.array(X_train).shape)  
    y_train = train_df.drop(['image_id','filename'],axis=1, inplace=False)
    print(y_train.head())
    y_train = np.array(y_train.values)
    X_train = np.array(X_train)
    sm = SMOTE(random_state=42)
    X_resampled, y_resampled = sm.fit_resample(X_train.reshape(X_train.shape[0], -1), y_train)

    X_resampled = X_resampled.reshape(-1, X_train.shape[1], X_train.shape[2], X_train.shape[3])
    
    return X_resampled, y_resampled

In [63]:
X_train,y_train=SMOTE_Data()
with open("training_data_smot_applied_500.pickle","wb") as f:
    pickle.dump((X_train,y_train),f)

(7200, 500, 500, 3)
           label
0        scratch
1           dent
2        scratch
3      tire flat
4  glass shatter


In [64]:
with open("training_data_smot_applied_500.pickle","rb") as f:
    (X_train,y_train)=pickle.load(f)

In [65]:
X_train.shape

(14094, 500, 500, 3)

In [66]:
labels=labels_df["label"].unique().tolist()

In [67]:
labels.sort()
print(labels)

['crack', 'dent', 'glass shatter', 'lamp broken', 'scratch', 'tire flat']


In [68]:
#Shuffling data points
from sklearn.utils import shuffle
X_train,y_train=shuffle(X_train,y_train,random_state=101)
print(X_train.shape,y_train.shape)

(14094, 500, 500, 3) (14094,)


In [69]:
y_train_new = []
for i in y_train:
    y_train_new.append(labels.index(i))
y_train = y_train_new
y_train = tf.keras.utils.to_categorical(y_train)

In [72]:
datagen = ImageDataGenerator(
    vertical_flip=True,
    rescale=1/255,
    validation_split=0.2
)

datagen.fit(X_train)
batch_size = 64
train_generator = datagen.flow(X_train, y_train, batch_size=batch_size,subset='training')
valid_generator=datagen.flow(X_train, y_train, batch_size=batch_size,subset='validation')

In [74]:
final_size=image_size

In [76]:
from tensorflow.keras.applications import InceptionV3

# Load InceptionV3 model with pre-trained weights
inception_model = InceptionV3(include_top=False, weights='imagenet', input_shape=(final_size, final_size, 3))

for layer in inception_model.layers:
    layer.trainable = True

In [26]:
inception_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_4 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d_94 (Conv2D)          (None, 111, 111, 32)         864       ['input_4[0][0]']             
                                                                                                  
 batch_normalization_94 (Ba  (None, 111, 111, 32)         96        ['conv2d_94[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 activation_94 (Activation)  (None, 111, 111, 32)         0         ['batch_normalizati

In [80]:
IModel = Sequential()
IModel.add(Input(shape=(final_size,final_size,3)))
IModel.add(inception_model)
IModel.add(Flatten())
IModel.add(Dense(64,activation="softplus"))
IModel.add(Dense(32,activation="softplus"))
IModel.add(Dense(6,activation="softmax"))
IModel.compile(loss='categorical_crossentropy',optimizer=tf.optimizers.Nadam(learning_rate=3e-5) ,metrics='accuracy')

Model_es = EarlyStopping(monitor = 'val_loss', patience = 8, verbose = 0)
Model_save = ModelCheckpoint(filepath = 'optimized_model_best_weights_resnet_tl_fifth.h5', monitor = 'val_accuracy', 
                      save_best_only = True, verbose = 1)
IModel.fit(train_generator,validation_data=valid_generator,batch_size=64,epochs=100,callbacks=[Model_es,Model_save])

Epoch 1/100
177/177 [==============================] - ETA: 0s - loss: 0.3222 - accuracy: 0.8857
Epoch 1: val_accuracy improved from -inf to 0.91199, saving model to optimized_model_best_weights_resnet_tl_fifth.h5
177/177 [==============================] - 137s 607ms/step - loss: 0.3222 - accuracy: 0.8857 - val_loss: 0.2282 - val_accuracy: 0.9120
Epoch 2/100
177/177 [==============================] - ETA: 0s - loss: 0.0765 - accuracy: 0.9741
Epoch 2: val_accuracy improved from 0.91199 to 0.94251, saving model to optimized_model_best_weights_resnet_tl_fifth.h5
177/177 [==============================] - 106s 597ms/step - loss: 0.0765 - accuracy: 0.9741 - val_loss: 0.1738 - val_accuracy: 0.9425
Epoch 3/100
177/177 [==============================] - ETA: 0s - loss: 0.0402 - accuracy: 0.9863
Epoch 3: val_accuracy improved from 0.94251 to 0.94819, saving model to optimized_model_best_weights_resnet_tl_fifth.h5
177/177 [==============================] - 106s 599ms/step - loss: 0.0402 - accura

In [83]:


IModel=tf.keras.models.load_model("optimized_model_best_weights_resnet_tl_fifth.h5")

test_data_images="test/images"
test_data_csv="test/test.csv"

test_data_df=pd.read_csv(test_data_csv)

test_data_df.head()

test_images=[]
predictions=[]
for index, row in test_data_df.iterrows():
    image_path = os.path.join(test_data_images,row['filename'])
    img = cv2.imread(image_path)
    img=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img=cv2.resize(img,(image_size, image_size))
    test_images.append(img/255.0)
    if (len(test_images))%32==0:
        xtest=np.array(test_images)
        predictions.extend(IModel.predict(xtest))
        test_images.clear()


predictions=[i.argmax() for i in predictions]


predictions=[labels[i] for i in predictions]

reverse_car_damage_type={v: k for k, v in car_damage_type.items()}
predictions=[reverse_car_damage_type[i] for i in predictions]

submission_df=pd.DataFrame({"image_id":test_data_df["image_id"].values,"label":predictions})




1/1 [==============================] - 0s 49ms/step


In [84]:
submission_df.to_csv("Ripik_submission_3_2.csv",index=False)

### Final training on full data

In [85]:
datagen = ImageDataGenerator(
    vertical_flip=True,
    horizontal_flip=True,
    rescale=1/255
)

datagen.fit(X_train)
batch_size = 64
train_generator = datagen.flow(X_train, y_train, batch_size=batch_size)
#valid_generator=datagen.flow(X_train, y_train, batch_size=batch_size,subset='validation')

In [86]:
from tensorflow.keras.applications import InceptionV3

# Load InceptionV3 model with pre-trained weights
inception_model = InceptionV3(include_top=False, weights='imagenet', input_shape=(final_size, final_size, 3))

for layer in inception_model.layers:
    layer.trainable = True

In [89]:
IModel = Sequential()
IModel.add(Input(shape=(final_size,final_size,3)))
IModel.add(inception_model)
IModel.add(Flatten())
IModel.add(Dense(128,activation="leaky_relu"))
IModel.add(Dense(32,activation="softplus"))
IModel.add(Dense(6,activation="softmax"))
IModel.compile(loss='categorical_crossentropy',optimizer=tf.optimizers.Adam(learning_rate=3e-5) ,metrics='accuracy')

#Model_es = EarlyStopping(monitor = 'val_loss', patience = 8, verbose = 0)
# #Model_save = ModelCheckpoint(filepath = 'optimized_model_best_weights_resnet_tl_fifth.h5', monitor = 'accuracy', 
#                       save_best_only = True, verbose = 1)
IModel.fit(train_generator,batch_size=64,epochs=25)

Epoch 1/25
221/221 [==============================] - 130s 499ms/step - loss: 0.1868 - accuracy: 0.9347
Epoch 2/25
221/221 [==============================] - 111s 503ms/step - loss: 0.0681 - accuracy: 0.9790
Epoch 3/25
221/221 [==============================] - 111s 501ms/step - loss: 0.0436 - accuracy: 0.9858
Epoch 4/25
221/221 [==============================] - 111s 502ms/step - loss: 0.0365 - accuracy: 0.9877
Epoch 5/25
221/221 [==============================] - 111s 500ms/step - loss: 0.0320 - accuracy: 0.9897
Epoch 6/25
221/221 [==============================] - 111s 499ms/step - loss: 0.0287 - accuracy: 0.9913
Epoch 7/25
221/221 [==============================] - 111s 502ms/step - loss: 0.0279 - accuracy: 0.9913
Epoch 8/25
221/221 [==============================] - 111s 502ms/step - loss: 0.0292 - accuracy: 0.9904
Epoch 9/25
221/221 [==============================] - 111s 500ms/step - loss: 0.0270 - accuracy: 0.9916
Epoch 10/25
221/221 [==============================] - 111s 501m

In [90]:

#IModel=tf.keras.models.load_model("optimized_model_best_weights_resnet_tl_fifth.h5")

test_data_images="test/images"
test_data_csv="test/test.csv"

test_data_df=pd.read_csv(test_data_csv)

test_data_df.head()

test_images=[]
predictions=[]
for index, row in test_data_df.iterrows():
    image_path = os.path.join(test_data_images,row['filename'])
    img = cv2.imread(image_path)
    img=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img=cv2.resize(img,(image_size, image_size))
    test_images.append(img/255.0)
    if (len(test_images))%32==0:
        xtest=np.array(test_images)
        predictions.extend(IModel.predict(xtest))
        test_images.clear()


predictions=[i.argmax() for i in predictions]


predictions=[labels[i] for i in predictions]

reverse_car_damage_type={v: k for k, v in car_damage_type.items()}
predictions=[reverse_car_damage_type[i] for i in predictions]

submission_df=pd.DataFrame({"image_id":test_data_df["image_id"].values,"label":predictions})

submission_df.to_csv("Ripik_final_submission.csv",index=False)
IModel.save('Ripik_final_model.h5')

1/1 [==============================] - 0s 53ms/step


/home/shahid/anaconda3/envs/tf_env/lib/python3.10/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
